In [4]:
#Cities data taken from - https://www.kaggle.com/blitzr/movehub-city-rankings   - cities.csv

#Using output files of above script
import pandas as pd
import numpy as np


data_folder = '/home/sidsvash26/kaggle_quora/data/'



In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

from subprocess import check_output
df = pd.read_csv(data_folder + 'train.csv')
locations = pd.read_csv(data_folder + 'cities.csv')

In [8]:
# There's lots of room to add more locations, but start with just countries
countries = set(locations['Country'].dropna(inplace=False).values.tolist())
all_places = countries

# Turn it into a Regex
regex = "|".join(sorted(set(all_places)))

In [9]:
from tqdm import tqdm

subset = df.shape[0] # Remove the subsetting 

results = []
print("processing:", df[0:subset].shape)
for index, row in tqdm(df[0:subset].iterrows()):
    q1 = str(row['question1'])
    q2 = str(row['question2'])

    rr = {}

    q1_matches = []
    q2_matches = []

    if (len(q1) > 0):
        q1_matches = [i.lower() for i in re.findall(regex, q1, flags=re.IGNORECASE)]

    if (len(q2) > 0):
        q2_matches = [i.lower() for i in re.findall(regex, q2, flags=re.IGNORECASE)]

    rr['z_q1_place_num'] = len(q1_matches)
    rr['z_q1_has_place'] =len(q1_matches) > 0

    rr['z_q2_place_num'] = len(q2_matches) 
    rr['z_q2_has_place'] = len(q2_matches) > 0

    rr['z_place_match_num'] = len(set(q1_matches).intersection(set(q2_matches)))
    rr['z_place_match'] = rr['z_place_match_num'] > 0

    rr['z_place_mismatch_num'] = len(set(q1_matches).difference(set(q2_matches)))
    rr['z_place_mismatch'] = rr['z_place_mismatch_num'] > 0

    results.append(rr)  
    


1it [00:00,  7.31it/s]

processing: (404290, 6)


404290it [03:19, 2024.38it/s]


In [10]:
out_df = pd.DataFrame.from_dict(results)

In [12]:
out_df.shape

(404290, 8)

In [13]:
out_df.head()


,z_place_match,z_place_match_num,z_place_mismatch,z_place_mismatch_num,z_q1_has_place,z_q1_place_num,z_q2_has_place,z_q2_place_num
0,False,0,True,1,True,1,False,0
1,False,0,False,0,False,0,True,1
2,False,0,False,0,False,0,False,0
3,False,0,False,0,False,0,False,0
4,False,0,False,0,False,0,False,0


In [16]:
#Build test features
df_test =pd.read_csv(data_folder + 'test.csv')

from tqdm import tqdm

subset = df_test.shape[0] # Remove the subsetting 

results = []
print("processing:", df_test[0:subset].shape)
for index, row in tqdm(df_test[0:subset].iterrows()):
    q1 = str(row['question1'])
    q2 = str(row['question2'])

    rr = {}

    q1_matches = []
    q2_matches = []

    if (len(q1) > 0):
        q1_matches = [i.lower() for i in re.findall(regex, q1, flags=re.IGNORECASE)]

    if (len(q2) > 0):
        q2_matches = [i.lower() for i in re.findall(regex, q2, flags=re.IGNORECASE)]

    rr['z_q1_place_num'] = len(q1_matches)
    rr['z_q1_has_place'] =len(q1_matches) > 0

    rr['z_q2_place_num'] = len(q2_matches) 
    rr['z_q2_has_place'] = len(q2_matches) > 0

    rr['z_place_match_num'] = len(set(q1_matches).intersection(set(q2_matches)))
    rr['z_place_match'] = rr['z_place_match_num'] > 0

    rr['z_place_mismatch_num'] = len(set(q1_matches).difference(set(q2_matches)))
    rr['z_place_mismatch'] = rr['z_place_mismatch_num'] > 0

    results.append(rr) 

0it [00:00, ?it/s]

processing: (2345796, 3)


2345796it [19:35, 1995.87it/s]


In [17]:
test_out_df = pd.DataFrame.from_dict(results)
print(out_df.shape)
print(test_out_df.shape)


(404290, 8)
(2345796, 8)


In [18]:
#Save features
#Saving features:
import pickle

train_X1 = np.array(out_df[['z_place_match_num', 'z_place_mismatch_num','z_q1_place_num','z_q2_place_num']])

test_X1 = np.array(test_out_df[['z_place_match_num', 'z_place_mismatch_num','z_q1_place_num','z_q2_place_num']])

pickle.dump(train_X1, open(data_folder + 'feats10_locations.sav', 'wb'))
pickle.dump(test_X1, open(data_folder + 'feats10_locations_for_test.sav', 'wb'))

In [19]:
#Check the dataset for specific examples:
df['mismatch'] = out_df['z_place_mismatch_num']

In [32]:
df[(df.mismatch>=1) & (df.is_duplicate ==1)].to_csv(data_folder + 'check_location_mismatch.csv')